In [1]:
# pip install pyspark

In [2]:
# pip install findspark

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

In [2]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Read CSV") \
    .config("spark.executor.memory", "18g") \
    .config("spark.driver.memory", "150g") \
    .getOrCreate()

# Read CSV into DataFrame
prices = spark.read.csv("itineraries.csv", header=True, inferSchema=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/09 16:04:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/09 16:04:21 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [12]:
delays = spark.read.csv("delays/raw/Flights_2018_1.csv", header=True, inferSchema=True)

In [3]:
delays.columns

NameError: name 'delays' is not defined

In [59]:
from pyspark.sql.functions import col

a = delays.filter(delays.DivArrDelay > 0)
a.select(col("DivArrDelay"), col("ArrDelay"), col("Operating_Airline ")).distinct().show(100)

+-----------+--------+------------------+
|DivArrDelay|ArrDelay|Operating_Airline |
+-----------+--------+------------------+
|      144.0|    NULL|                OO|
|      137.0|    NULL|                EV|
|      237.0|    NULL|                OO|
|      118.0|    NULL|                EV|
|       89.0|    NULL|                OO|
|      195.0|    NULL|                OO|
|      278.0|    NULL|                B6|
|      569.0|    NULL|                HA|
|      135.0|    NULL|                B6|
|      167.0|    NULL|                B6|
|      154.0|    NULL|                OO|
|      262.0|    NULL|                OO|
|      943.0|    NULL|                9E|
|      421.0|    NULL|                OO|
|      123.0|    NULL|                OO|
|      197.0|    NULL|                G4|
|       88.0|    NULL|                B6|
|      137.0|    NULL|                OO|
|      383.0|    NULL|                9E|
|      229.0|    NULL|                OO|
|      945.0|    NULL|            

In [4]:
from pyspark.sql.functions import year

# Extract year from fightDate column and get distinct years
unique_years = delays.select(year("FlightDate").alias("year")).distinct()

# Show unique years
unique_years.show()


+----+
|year|
+----+
|2018|
|2019|
|2020|
|2021|
|2022|
+----+



In [4]:
# prices.limit(5).toPandas().head()

- If the record for a particular flight is not present for a date -> Our assumption is that the flight did not run during those dates

In [5]:
delays.limit(5).toPandas().head()

24/04/27 13:39:36 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2018-01-23,Endeavor Air Inc.,ABY,ATL,False,False,1202,1157.0,0.0,-5.0,...,1211.0,1249.0,7.0,1304,-8.0,0.0,-1.0,1300-1359,1,0.0
1,2018-01-24,Endeavor Air Inc.,ABY,ATL,False,False,1202,1157.0,0.0,-5.0,...,1210.0,1246.0,12.0,1304,-6.0,0.0,-1.0,1300-1359,1,0.0
2,2018-01-25,Endeavor Air Inc.,ABY,ATL,False,False,1202,1153.0,0.0,-9.0,...,1211.0,1251.0,11.0,1304,-2.0,0.0,-1.0,1300-1359,1,0.0
3,2018-01-26,Endeavor Air Inc.,ABY,ATL,False,False,1202,1150.0,0.0,-12.0,...,1207.0,1242.0,11.0,1304,-11.0,0.0,-1.0,1300-1359,1,0.0
4,2018-01-27,Endeavor Air Inc.,ABY,ATL,False,False,1400,1355.0,0.0,-5.0,...,1412.0,1448.0,11.0,1500,-1.0,0.0,-1.0,1500-1559,1,0.0


In [6]:
from pyspark.sql.functions import col

negative_delay_count = delays.filter(col('DepDelay') < 0).count()
positive_delay_count = delays.filter(col('DepDelay') > 0).count()
no_delay_count = delays.filter(col('DepDelay') == 0).count()


print(f"Flights that started earlier than scheduled: {negative_delay_count}")
print(f"Flights that started later than scheduled: {positive_delay_count}")
print(f"Flights that started as per schedule: {no_delay_count}")


Flights that started earlier than scheduled: 17871985
Flights that started later than scheduled: 9184614
Flights that started as per schedule: 1374099


<h4> We are only concerned with the flights that are departure delayed.So the flights which have departed earlier or on time -> Makes no difference to us </h4>

In [7]:
delays.select("FlightDate").limit(5).show()

+----------+
|FlightDate|
+----------+
|2018-01-23|
|2018-01-24|
|2018-01-25|
|2018-01-26|
|2018-01-27|
+----------+



In [8]:
delays.columns

['FlightDate',
 'Airline',
 'Origin',
 'Dest',
 'Cancelled',
 'Diverted',
 'CRSDepTime',
 'DepTime',
 'DepDelayMinutes',
 'DepDelay',
 'ArrTime',
 'ArrDelayMinutes',
 'AirTime',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'Distance',
 'Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'Marketing_Airline_Network',
 'Operated_or_Branded_Code_Share_Partners',
 'DOT_ID_Marketing_Airline',
 'IATA_Code_Marketing_Airline',
 'Flight_Number_Marketing_Airline',
 'Operating_Airline',
 'DOT_ID_Operating_Airline',
 'IATA_Code_Operating_Airline',
 'Tail_Number',
 'Flight_Number_Operating_Airline',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRS

In [10]:
from pyspark.sql.functions import when

non_negative_delays = delays.withColumn('DepDelay', when(col('DepDelay') < 0, 0).otherwise(col('DepDelay')))

In [11]:
zero_delay_count = non_negative_delays.filter(col('DepDelay') == 0).count()

print(f"Flights that weren't delayed: {zero_delay_count}")
print(f"Flights that were delayed: {positive_delay_count}")

Flights that weren't delayed: 19246084
Flights that were delayed: 9184614


In [12]:
from pyspark.sql.functions import weekofyear, to_date

# Extract year from fightDate column and get distinct years
unique_weeks = non_negative_delays.select(weekofyear("FlightDate").alias("week")).distinct()

# Show unique years
unique_weeks.show()

# from pyspark.sql.functions import date_format, date_add
# week_number = df.select(date_format("fightDate", "w").alias("week_number"))


+----+
|week|
+----+
|   1|
|   3|
|   5|
|   4|
|   2|
|  44|
|  40|
|  41|
|  43|
|  42|
|  47|
|  48|
|  45|
|  46|
|  52|
|  49|
|  51|
|  50|
|   6|
|   9|
+----+
only showing top 20 rows



In [13]:
non_negative_delays = non_negative_delays.withColumn("FlightYear", year("FlightDate"))

In [14]:
non_negative_delays = non_negative_delays.withColumn("FlightWeek", weekofyear("FlightDate"))

In [15]:
non_negative_delays.limit(10).toPandas()

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings,FlightYear,FlightWeek
0,2018-01-23,Endeavor Air Inc.,ABY,ATL,False,False,1202,1157.0,0.0,0.0,...,7.0,1304,-8.0,0.0,-1.0,1300-1359,1,0.0,2018,4
1,2018-01-24,Endeavor Air Inc.,ABY,ATL,False,False,1202,1157.0,0.0,0.0,...,12.0,1304,-6.0,0.0,-1.0,1300-1359,1,0.0,2018,4
2,2018-01-25,Endeavor Air Inc.,ABY,ATL,False,False,1202,1153.0,0.0,0.0,...,11.0,1304,-2.0,0.0,-1.0,1300-1359,1,0.0,2018,4
3,2018-01-26,Endeavor Air Inc.,ABY,ATL,False,False,1202,1150.0,0.0,0.0,...,11.0,1304,-11.0,0.0,-1.0,1300-1359,1,0.0,2018,4
4,2018-01-27,Endeavor Air Inc.,ABY,ATL,False,False,1400,1355.0,0.0,0.0,...,11.0,1500,-1.0,0.0,-1.0,1500-1559,1,0.0,2018,4
5,2018-01-28,Endeavor Air Inc.,ABY,ATL,False,False,1202,1202.0,NaN,NaN,...,13.0,1304,22.0,1.0,1.0,1300-1359,1,0.0,2018,4
6,2018-01-29,Endeavor Air Inc.,ABY,ATL,False,False,1202,1204.0,2.0,2.0,...,10.0,1304,-1.0,0.0,-1.0,1300-1359,1,0.0,2018,5
7,2018-01-30,Endeavor Air Inc.,ABY,ATL,False,False,1202,1153.0,0.0,0.0,...,11.0,1304,-9.0,0.0,-1.0,1300-1359,1,0.0,2018,5
8,2018-01-31,Endeavor Air Inc.,ABY,ATL,False,False,1202,1153.0,0.0,0.0,...,8.0,1304,NaN,NaN,NaN,1300-1359,1,0.0,2018,5
9,2018-01-03,Endeavor Air Inc.,ATL,ABY,False,False,1037,1101.0,24.0,24.0,...,3.0,1137,22.0,1.0,1.0,1100-1159,1,0.0,2018,1


In [16]:
non_negative_delays.groupBy("FlightYear", "FlightWeek").count().show()

+----------+----------+------+
|FlightYear|FlightWeek| count|
+----------+----------+------+
|      2018|         1|122773|
|      2018|         3|103075|
|      2018|         5| 82196|
|      2018|         2|101734|
|      2018|         4|103148|
|      2018|        41|152788|
|      2018|        43|151973|
|      2018|        40|151979|
|      2018|        44|142823|
|      2018|        42|153135|
|      2018|        48|148387|
|      2018|        47|143141|
|      2018|        45|149965|
|      2018|        46|152732|
|      2018|        50|146117|
|      2018|        51|154601|
|      2018|        49|145884|
|      2018|        52|146232|
|      2018|         8| 68695|
|      2018|         9| 72355|
+----------+----------+------+
only showing top 20 rows



In [17]:
non_negative_delays.groupBy("FlightYear", "FlightWeek").count().orderBy("FlightYear", "FlightWeek").show(300)

+----------+----------+------+
|FlightYear|FlightWeek| count|
+----------+----------+------+
|      2018|         1|122773|
|      2018|         2|101734|
|      2018|         3|103075|
|      2018|         4|103148|
|      2018|         5| 82196|
|      2018|         6| 65919|
|      2018|         7| 66881|
|      2018|         8| 68695|
|      2018|         9| 72355|
|      2018|        10| 77253|
|      2018|        11| 79319|
|      2018|        12| 79347|
|      2018|        13| 83885|
|      2018|        14|112488|
|      2018|        15|112565|
|      2018|        16|112597|
|      2018|        17|112482|
|      2018|        18| 84008|
|      2018|        19| 78965|
|      2018|        20| 79228|
|      2018|        21| 78375|
|      2018|        22| 76092|
|      2018|        23| 73560|
|      2018|        24| 74837|
|      2018|        25| 75049|
|      2018|        26| 76329|
|      2018|        27| 79549|
|      2018|        28| 83949|
|      2018|        29| 83840|
|      2

- Flight data from weeks 31 to 51 in 2022 is not present

In [18]:
non_negative_delays.select(col("Airline")).distinct().show(30)

+--------------------+
|             Airline|
+--------------------+
|GoJet Airlines, L...|
|   Endeavor Air Inc.|
|       Allegiant Air|
|SkyWest Airlines ...|
|      Virgin America|
|         Horizon Air|
|United Air Lines ...|
|Air Wisconsin Air...|
|Trans States Airl...|
|    Compass Airlines|
|Peninsula Airways...|
|Frontier Airlines...|
|            Cape Air|
|Southwest Airline...|
|ExpressJet Airlin...|
|     JetBlue Airways|
|Commutair Aka Cha...|
|Empire Airlines Inc.|
|Hawaiian Airlines...|
|Alaska Airlines Inc.|
|Delta Air Lines Inc.|
|  Mesa Airlines Inc.|
|   Republic Airlines|
|    Spirit Air Lines|
|         Comair Inc.|
|           Envoy Air|
|Capital Cargo Int...|
|American Airlines...|
+--------------------+



- 28 distinct flights

In [19]:
non_negative_delays.limit(10).toPandas()

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings,FlightYear,FlightWeek
0,2018-01-23,Endeavor Air Inc.,ABY,ATL,False,False,1202,1157.0,0.0,0.0,...,7.0,1304,-8.0,0.0,-1.0,1300-1359,1,0.0,2018,4
1,2018-01-24,Endeavor Air Inc.,ABY,ATL,False,False,1202,1157.0,0.0,0.0,...,12.0,1304,-6.0,0.0,-1.0,1300-1359,1,0.0,2018,4
2,2018-01-25,Endeavor Air Inc.,ABY,ATL,False,False,1202,1153.0,0.0,0.0,...,11.0,1304,-2.0,0.0,-1.0,1300-1359,1,0.0,2018,4
3,2018-01-26,Endeavor Air Inc.,ABY,ATL,False,False,1202,1150.0,0.0,0.0,...,11.0,1304,-11.0,0.0,-1.0,1300-1359,1,0.0,2018,4
4,2018-01-27,Endeavor Air Inc.,ABY,ATL,False,False,1400,1355.0,0.0,0.0,...,11.0,1500,-1.0,0.0,-1.0,1500-1559,1,0.0,2018,4
5,2018-01-28,Endeavor Air Inc.,ABY,ATL,False,False,1202,1202.0,NaN,NaN,...,13.0,1304,22.0,1.0,1.0,1300-1359,1,0.0,2018,4
6,2018-01-29,Endeavor Air Inc.,ABY,ATL,False,False,1202,1204.0,2.0,2.0,...,10.0,1304,-1.0,0.0,-1.0,1300-1359,1,0.0,2018,5
7,2018-01-30,Endeavor Air Inc.,ABY,ATL,False,False,1202,1153.0,0.0,0.0,...,11.0,1304,-9.0,0.0,-1.0,1300-1359,1,0.0,2018,5
8,2018-01-31,Endeavor Air Inc.,ABY,ATL,False,False,1202,1153.0,0.0,0.0,...,8.0,1304,NaN,NaN,NaN,1300-1359,1,0.0,2018,5
9,2018-01-03,Endeavor Air Inc.,ATL,ABY,False,False,1037,1101.0,24.0,24.0,...,3.0,1137,22.0,1.0,1.0,1100-1159,1,0.0,2018,1


In [20]:
delays.select(col("Diverted")).count()

29193782

In [21]:
delays.where(col("Cancelled") == True).count()

777267

In [22]:
from pyspark.sql.functions import col, count, when

flight_counts = delays.groupBy('Airline') \
                  .agg(count('*').alias('Airline Count'),
                       count(when(col('Cancelled') == True, True)).alias('Cancelled Count'))
flight_counts_ratio = flight_counts.withColumn('Cancelled Ratio', col('Cancelled Count') / col('Airline Count'))
flight_counts_ratio.show(30)

+--------------------+-------------+---------------+--------------------+
|             Airline|Airline Count|Cancelled Count|     Cancelled Ratio|
+--------------------+-------------+---------------+--------------------+
|GoJet Airlines, L...|       276486|           9240| 0.03341941364119702|
|   Endeavor Air Inc.|       998224|          20193|0.020228926573594704|
|       Allegiant Air|       489400|          22409|0.045788720882713525|
|SkyWest Airlines ...|      3159683|          72587|0.022972874177567812|
|      Virgin America|        17670|            433|  0.0245048104131296|
|         Horizon Air|       471153|          11022|0.023393674666191237|
|United Air Lines ...|      2354538|          47061|0.019987360577743914|
|Air Wisconsin Air...|       380202|          14092| 0.03706450781426715|
|Trans States Airl...|       161590|           6614| 0.04093075066526394|
|    Compass Airlines|       154985|           1986|0.012814143304190728|
|Peninsula Airways...|         2783|  

In [23]:
print(list(non_negative_delays.columns))

['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted', 'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime', 'ArrDelayMinutes', 'AirTime', 'CRSElapsedTime', 'ActualElapsedTime', 'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners', 'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline', 'Flight_Number_Marketing_Airline', 'Operating_Airline', 'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline', 'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName', 'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac', 'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName', 'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrDelay', 'ArrDel15', 'ArrivalDelayGroups'

In [24]:
# # Can be used in output
# Cancelled, Diverted


# ArrDelay


In [25]:
# # Required 
# Airline, Origin, Dest,  FlightYear, FlightWeek

# # optiional
# CRSDepTime - dayPart conversion: By default search for all
# Distance - Computed by org and dest
# CRSArrTime - dayPart conversion: By default search for all

In [26]:
# # Not required
# FlightDate, Cancelled, Diverted, DepDelay, CRSDepTime, DepTime, DepDelay, ArrTime, ArrDelayMinutes, DepDelayMinutes

In [27]:
# required_fields_df = non_negative_delays[['Airline', 'Origin', 'Dest',  'FlightYear', 'FlightWeek', 'DayOfWeek', 'Distance', 'CRSDepTime', 'CRSArrTime', 'ArrDelay']]
required_fields_df = non_negative_delays[['Airline', 'Origin', 'Dest',  'FlightYear', 'FlightWeek', 'Distance', 'CRSDepTime', 'CRSArrTime', 'ArrDelay']]
test = required_fields_df

In [28]:
test.show(100)

+-----------------+------+----+----------+----------+--------+----------+----------+--------+
|          Airline|Origin|Dest|FlightYear|FlightWeek|Distance|CRSDepTime|CRSArrTime|ArrDelay|
+-----------------+------+----+----------+----------+--------+----------+----------+--------+
|Endeavor Air Inc.|   ABY| ATL|      2018|         4|   145.0|      1202|      1304|    -8.0|
|Endeavor Air Inc.|   ABY| ATL|      2018|         4|   145.0|      1202|      1304|    -6.0|
|Endeavor Air Inc.|   ABY| ATL|      2018|         4|   145.0|      1202|      1304|    -2.0|
|Endeavor Air Inc.|   ABY| ATL|      2018|         4|   145.0|      1202|      1304|   -11.0|
|Endeavor Air Inc.|   ABY| ATL|      2018|         4|   145.0|      1400|      1500|    -1.0|
|Endeavor Air Inc.|   ABY| ATL|      2018|         4|   145.0|      1202|      1304|    22.0|
|Endeavor Air Inc.|   ABY| ATL|      2018|         5|   145.0|      1202|      1304|    -1.0|
|Endeavor Air Inc.|   ABY| ATL|      2018|         5|   145.

In [29]:
required_fields_df.select(col("Origin")).distinct().count()

388

In [30]:
required_fields_df.select(col("Dest")).distinct().count()

388

In [31]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline


indexers = [StringIndexer(inputCol=column, outputCol=column+"_index", handleInvalid="keep") for column in ["Airline", "Origin", "Dest"]]

# Perform OneHotEncoding on the indexed columns
encoders = [OneHotEncoder(inputCol=column+"_index", outputCol=column+"_onehot") for column in ["Airline", "Origin", "Dest"]]

# Create a pipeline to execute the indexers and encoders sequentially
pipeline = Pipeline(stages=indexers + encoders)

# Fit the pipeline to the data and transform the DataFrame
model = pipeline.fit(test)
encoded_df = model.transform(test)

# Show the encoded DataFrame
encoded_df.show()


+-----------------+------+----+----------+----------+--------+----------+----------+--------+-------------+------------+----------+--------------+-----------------+-----------------+
|          Airline|Origin|Dest|FlightYear|FlightWeek|Distance|CRSDepTime|CRSArrTime|ArrDelay|Airline_index|Origin_index|Dest_index|Airline_onehot|    Origin_onehot|      Dest_onehot|
+-----------------+------+----+----------+----------+--------+----------+----------+--------+-------------+------------+----------+--------------+-----------------+-----------------+
|Endeavor Air Inc.|   ABY| ATL|      2018|         4|   145.0|      1202|      1304|    -8.0|          8.0|       275.0|       1.0|(28,[8],[1.0])|(388,[275],[1.0])|  (388,[1],[1.0])|
|Endeavor Air Inc.|   ABY| ATL|      2018|         4|   145.0|      1202|      1304|    -6.0|          8.0|       275.0|       1.0|(28,[8],[1.0])|(388,[275],[1.0])|  (388,[1],[1.0])|
|Endeavor Air Inc.|   ABY| ATL|      2018|         4|   145.0|      1202|      1304| 

In [62]:
encoded_df.select('Dest_onehot').show()

+-----------------+
|      Dest_onehot|
+-----------------+
|  (388,[1],[1.0])|
|  (388,[1],[1.0])|
|  (388,[1],[1.0])|
|  (388,[1],[1.0])|
|  (388,[1],[1.0])|
|  (388,[1],[1.0])|
|  (388,[1],[1.0])|
|  (388,[1],[1.0])|
|(388,[274],[1.0])|
|(388,[274],[1.0])|
|(388,[274],[1.0])|
|(388,[274],[1.0])|
|(388,[274],[1.0])|
|(388,[274],[1.0])|
|(388,[274],[1.0])|
|(388,[274],[1.0])|
|(388,[274],[1.0])|
|(388,[274],[1.0])|
|(388,[274],[1.0])|
|(388,[274],[1.0])|
+-----------------+
only showing top 20 rows



In [35]:
encoded_df.printSchema

<bound method DataFrame.printSchema of DataFrame[Airline: string, Origin: string, Dest: string, FlightYear: int, FlightWeek: int, Distance: double, CRSDepTime: int, CRSArrTime: int, ArrDelay: double, Airline_index: double, Origin_index: double, Dest_index: double, Airline_onehot: vector, Origin_onehot: vector, Dest_onehot: vector]>

In [36]:
encoded_df.columns

['Airline',
 'Origin',
 'Dest',
 'FlightYear',
 'FlightWeek',
 'Distance',
 'CRSDepTime',
 'CRSArrTime',
 'ArrDelay',
 'Airline_index',
 'Origin_index',
 'Dest_index',
 'Airline_onehot',
 'Origin_onehot',
 'Dest_onehot']

In [37]:
encoded_df = encoded_df.drop('Airline', 'Origin', 'Dest', "FlightYear", "CRSDepTime", "CRSArrTime")

In [38]:
# encoded_df.toPandas().head(10)

In [39]:
encoded_df.columns

['FlightWeek',
 'Distance',
 'ArrDelay',
 'Airline_index',
 'Origin_index',
 'Dest_index',
 'Airline_onehot',
 'Origin_onehot',
 'Dest_onehot']

In [40]:
encoded_df = encoded_df.dropna()


In [41]:
encoded_df.show()

+----------+--------+--------+-------------+------------+----------+--------------+-----------------+-----------------+
|FlightWeek|Distance|ArrDelay|Airline_index|Origin_index|Dest_index|Airline_onehot|    Origin_onehot|      Dest_onehot|
+----------+--------+--------+-------------+------------+----------+--------------+-----------------+-----------------+
|         4|   145.0|    -8.0|          8.0|       275.0|       1.0|(28,[8],[1.0])|(388,[275],[1.0])|  (388,[1],[1.0])|
|         4|   145.0|    -6.0|          8.0|       275.0|       1.0|(28,[8],[1.0])|(388,[275],[1.0])|  (388,[1],[1.0])|
|         4|   145.0|    -2.0|          8.0|       275.0|       1.0|(28,[8],[1.0])|(388,[275],[1.0])|  (388,[1],[1.0])|
|         4|   145.0|   -11.0|          8.0|       275.0|       1.0|(28,[8],[1.0])|(388,[275],[1.0])|  (388,[1],[1.0])|
|         4|   145.0|    -1.0|          8.0|       275.0|       1.0|(28,[8],[1.0])|(388,[275],[1.0])|  (388,[1],[1.0])|
|         4|   145.0|    22.0|          

In [42]:
encoded_df.repartition(125, "Airline")

DataFrame[FlightWeek: int, Distance: double, ArrDelay: double, Airline_index: double, Origin_index: double, Dest_index: double, Airline_onehot: vector, Origin_onehot: vector, Dest_onehot: vector]

In [43]:
encoded_df.count()

28347599

In [44]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator


# Combine features into a single vector column
assembler = VectorAssembler(inputCols=["FlightWeek", "Distance", "Airline_onehot",	"Origin_onehot",	"Dest_onehot"],
                            outputCol="features")
output = assembler.transform(encoded_df)



total_count = output.count()
train_count = int(total_count * 0.7)

train_data = output.limit(train_count)
test_data = output.subtract(train_data)


# Create a RandomForestRegressor
rf = RandomForestRegressor(featuresCol="features", labelCol="ArrDelay", numTrees=10)

# Train the model
model = rf.fit(train_data)

# Make predictions
predictions = model.transform(test_data)

# Evaluate the model
evaluator = RegressionEvaluator(labelCol="ArrDelay", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)



Root Mean Squared Error (RMSE) on test data = 74.5586


In [45]:
from pyspark.sql.functions import abs

# Calculate Mean Absolute Error (MAE)
mae = predictions.select(abs(predictions["ArrDelay"] - predictions["prediction"]).alias("error")).agg({"error": "mean"}).collect()[0][0]

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
print("Mean Absolute Error (MAE) on test data = %g" % mae)


Root Mean Squared Error (RMSE) on test data = 74.5586
Mean Absolute Error (MAE) on test data = 34.3483


In [46]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import abs
from pyspark.ml.evaluation import RegressionEvaluator

# Combine features into a single vector column
assembler = VectorAssembler(inputCols=["FlightWeek", "Distance", "Airline_onehot", "Origin_onehot", "Dest_onehot"],
                            outputCol="features")
output = assembler.transform(encoded_df)

# Split the data into training and test sets
total_count = output.count()
train_count = int(total_count * 0.7)

train_data = output.limit(train_count)
test_data = output.subtract(train_data)


# Create a LinearRegression model
lr = LinearRegression(featuresCol="features", labelCol="ArrDelay")

# Fit the model
lr_model = lr.fit(train_data)

# Make predictions
predictions = lr_model.transform(test_data)

# Evaluate the model
evaluator = RegressionEvaluator(labelCol="ArrDelay", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


24/04/25 15:31:38 WARN Instrumentation: [abd8c62a] regParam is zero, which might cause numerical instability and overfitting.
24/04/25 15:31:46 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/04/25 15:31:46 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/04/25 15:31:46 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
24/04/25 15:31:46 WARN Instrumentation: [abd8c62a] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


Root Mean Squared Error (RMSE) on test data = 74.5118


In [47]:
from pyspark.sql.functions import abs

# Calculate Mean Absolute Error (MAE)
mae = predictions.select(abs(predictions["ArrDelay"] - predictions["prediction"]).alias("error")).agg({"error": "mean"}).collect()[0][0]

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
print("Mean Absolute Error (MAE) on test data = %g" % mae)


Root Mean Squared Error (RMSE) on test data = 74.5118
Mean Absolute Error (MAE) on test data = 34.3847


In [46]:
output_df = non_negative_delays[['ArrDelay']]

In [47]:
additional_output_df = non_negative_delays[['Cancelled', 'Diverted']]

In [48]:
# from pyspark.ml.feature import VectorAssembler, StandardScaler
# from pyspark.ml.feature import PCA

# # Assemble feature vectors
# assembler = VectorAssembler(inputCols=["feature1", "feature2", "feature3"], outputCol="features")
# df = assembler.transform(df)

# # Standardize features
# scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
# scaler_model = scaler.fit(df)
# df = scaler_model.transform(df)

# # Fit PCA model
# pca = PCA(k=2, inputCol="scaled_features", outputCol="pca_features")
# model = pca.fit(df)

# # Transform data
# df_pca = model.transform(df)

# # Show PCA features
# df_pca.select("pca_features").show(truncate=False)

In [49]:
pwd

'/Users/shreyasmac/Documents/Notes/Big Data/Final Project'